In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, explode, split, count, col
from pyspark.sql.types import StructType, StructField, StringType

#необходимая библиотека для интеграции Spark и Postgres
spark_jars_packages = ",".join(
        [
            "org.postgresql:postgresql:42.4.0",
            'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0'
        ]
    )

In [2]:
spark = SparkSession.builder \
    .appName("read kafka topic") \
    .config("spark.jars.packages", spark_jars_packages) \
    .getOrCreate()

23/02/02 21:48:17 WARN Utils: Your hostname, fhmdrmaiepeds2jrv5bm resolves to a loopback address: 127.0.1.1; using 10.128.0.17 instead (on interface eth0)
23/02/02 21:48:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b949c6a-923e-4de1-acec-978385216770;1.0
	confs: [default]
	found org.postgresql#postgresql;42.4.0 in central
	found org.checkerframework#checker-qual;3.5.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-log

23/02/02 21:48:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read \
    .format('kafka') \
    .option('-b', 'kafka.bootstrap.servers') \
    .option('-X security.protocol', 'kafka.security.protocol') \
    .option('-X sasl.mechanisms', 'kafka.sasl.mechanism') \
    .option('kafka.sasl.mechanism', 'org.apache.kafka.common.security.scram.ScramLoginModule required username=\"de-student\" password=\"ltcneltyn\";') \
    .option('-t', 'persist_topic') \
    .option("kafka.metadata.max.age.ms", "1000") \
    .load()

IllegalArgumentException: One of the following options must be specified for Kafka source: subscribe, subscribepattern, assign. See the docs for more details.

In [17]:
#создаём SparkSession и передаём библиотеку для работы с Postgres
spark = SparkSession.builder \
    .appName("join data") \
    .config("spark.jars.packages", spark_jars_packages) \
    .getOrCreate()

In [18]:
#вычитываем данные из таблицы
tableDF = spark.read \
                    .format('jdbc') \
                    .option('url', 'jdbc:postgresql://rc1a-fswjkpli01zafgjm.mdb.yandexcloud.net:6432/de') \
                    .option('driver', 'org.postgresql.Driver') \
                    .option('dbtable', 'words') \
                    .option('user', 'student') \
                    .option('password', 'de-student') \
                    .load()

In [64]:
tableDF.show()

+---+---------+
| id|    words|
+---+---------+
|  1|    spark|
|  2|      sql|
|  3|     core|
|  4|streaming|
+---+---------+



In [19]:
#определяем схему для датафрейма
userSchema = StructType([StructField("text", StringType(), True)])

In [28]:
#читаем текст из файла
wordsDF = spark.readStream.schema(userSchema).format('text').load('/datas8')

In [53]:
splitWordsDF = wordsDF \
    .select(explode(split('text', ',')).alias('words') )

In [63]:
wordCountsDF = splitWordsDF \
    .groupBy('words') \
    .count()

In [62]:
wordCountsDF.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() \
    .awaitTermination() 

23/01/28 17:47:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-953583b4-b365-4b85-8274-cf9055ffe89d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/28 17:47:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+-----+
|         words|count|
+--------------+-----+
|spark context.|    1|
|          core|    1|
|     streaming|    2|
|           rdd|    1|
|           sql|    1|
| spark session|    1|
|       dataset|    1|
|         spark|    4|
|       pyspark|    1|
|    structured|    1|
|     dataframe|    1|
+--------------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [94]:
joinDF = wordCountsDF \
    .join(tableDF, ['words'], 'left') \
    .filter(col('id').isNull()) \
    .select('words')


In [95]:
joinDF.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() \
    .awaitTermination() 

23/01/28 18:12:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2d7d38f5-56b7-4a27-b0d5-fb53bdfc63a0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/28 18:12:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+
|         words|
+--------------+
|spark context.|
|           rdd|
| spark session|
|       dataset|
|       pyspark|
|    structured|
|     dataframe|
+--------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, TimestampType, LongType


spark_jars_packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0"

config = {
    'kafka.bootstrap.servers': 'rc1b-2erh7b35n4j4v869.mdb.yandexcloud.net:9091',
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'SCRAM-SHA-512',
    'kafka.sasl.jaas.config': 'org.apache.kafka.common.security.scram.ScramLoginModule required username="de-student" password="ltcneltyn";',
    "subscribe": 'student.topic.cohort5.agredyaev'
}


def spark_init() -> SparkSession:
    return SparkSession.builder \
        .appName("read kafka topic") \
        .config("spark.jars.packages", spark_jars_packages) \
        .getOrCreate()


def load_df(spark: SparkSession) -> DataFrame:
    return spark.readStream \
        .format('kafka') \
        .options(**config) \
        .load()


def transform(df: DataFrame) -> DataFrame:

    schema = StructType([

        StructField("lat", DoubleType()),
        StructField("timestamp", DoubleType()),
        StructField("lon", DoubleType()),
        StructField("client_id", StringType())
    ])

    return df \
        .withColumn('key', F.col('key').cast(StringType())) \
        .withColumn('value', F.col('value').cast(StringType())) \
        .withColumn('event', F.from_json('value', schema=schema)) \
        .select('event.*') \
        .withColumn('timestamp', F.from_unixtime(F.col('timestamp'), "yyyy-MM-dd' 'HH:mm:ss.SSS").cast(TimestampType())) \
        .drop_duplicates(['client_id', 'timestamp'])


def write_stream(df):
    return df \
        .writeStream \
        .outputMode("append") \
        .format("console") \
        .option("truncate", False) \
        .trigger(once=True) \
        .start()


In [6]:
spark = spark_init()
source_df = load_df(spark)
output_df = transform(source_df)

query = write_stream(output_df)
try:
    query.awaitTermination()
finally:
    query.stop()

23/02/04 11:01:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-16294063-5839-4676-a87a-492ac9130e04. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/04 11:01:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-------------------+---------+------------------------------------+
|lat      |timestamp          |lon      |client_id                           |
+---------+-------------------+---------+------------------------------------+
|55.679379|2023-02-04 11:01:17|37.510885|3fea6e3a-a47b-11ed-9aeb-d00d6d050095|
|55.735252|2023-02-04 11:01:17|37.573233|3ff5aeee-a47b-11ed-9aeb-d00d6d050095|
+---------+-------------------+---------+------------------------------------+

